# Delta Lake write example

This walkthrough mirrors the Parquet example but targets Delta Lake storage. It writes tables to a local warehouse and inspects recent commits via the Delta transaction log.

In [ ]:
import datetime
from pathlib import Path
import importlib
from dataset_generator import create_generator, create_writer, WriterOptions, write_dataset
if importlib.util.find_spec("deltalake") is None:
    raise RuntimeError("Install dataset-generator[delta] before running this notebook.")

## Configure generator

We reuse the ecommerce generator with modest scale so Delta commits complete quickly.

In [ ]:
generator = create_generator("ecommerce",
    seed=11,
    n_customers=400,
    n_products=150,
    orders_per_day=90,
    order_items_mean=2.2,
    file_rows_target=150,
    start_date=datetime.date(2023, 2, 1),
    end_date=datetime.date(2023, 2, 5),
)

## Write Delta tables

The writer creates the Delta log inside `examples/demo_output/delta`. Subsequent runs append to the same tables, producing additional commits.

In [ ]:
warehouse = Path("examples/demo_output/delta").resolve()
writer = create_writer("delta", str(warehouse), s3=None, catalog=None, options=WriterOptions(file_rows_target=150))
write_dataset(generator, writer)
sorted(path.relative_to(warehouse.parent) for path in warehouse.rglob("*.json"))[:5]

## Inspect commit history

Use the Delta Lake Python API to confirm the table was created and inspect recent operations.

In [ ]:
from deltalake import DeltaTable
table = DeltaTable(str(warehouse / "orders"))
table.history(limit=5)